In [ ]:
# !pip install llama-index-llms-gemini llama-index

In [ ]:
# !pip install python-dotenv

In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

# Get the API key
api_key = os.getenv("GEMINI_API_KEY")

# print("Your API Key:", api_key)  # Ensure it loads correctly


In [ ]:
from llama_index.llms.gemini import Gemini

class Generators:
    def __init__(self, model="models/gemini-2.0-flash"):
        self.llm = Gemini(
            model=model,
            api_key=api_key,
        )

    def get_llm(self):
        return self.llm

In [4]:
import math

session = dict()

def multiply(a: float, b: float) -> float:
    """useful tool to Multiply two numbers and returns the product
    make sure that both numbers a, b are real numbers and if not try to convert them to real numbers
    args:
        a: float
        b: float
    requires:
        only_real_numbers
    """
    return float(a) * float(b)

def add(a: float, b: float) -> float:
    """useful tool to Add two numbers and returns the sum,
    make sure that both numbers a, b are real numbers and if not try to convert them to real numbers
    args:
        a: float
        b: float

    requires:
        only_real_numbers
    """
    return float(a) + float(b)

def only_real_numbers(a: float) -> float:
    """ useful tool to verify if the number is real"""
    try:
        float(a)
    except ValueError:
        return f"{a} is not a real number so try converting it to a float and try again"

In [5]:
from llama_index.core.agent import FunctionCallingAgent
from llama_index.core.tools import FunctionTool

add_tool = FunctionTool.from_defaults(fn=add)
multiply_tool = FunctionTool.from_defaults(fn=multiply)
real_number_tool = FunctionTool.from_defaults(fn=only_real_numbers)

class AgentController:
    def __init__(self):
        self.llm = Generators().get_llm()
        self.system_prompt = """
                                You are a math expert. You will only use the tools available to you.
                                IMPORTANT NOTE: You will ALWAYS evaluate the user's query and perfom query classification and provide three things:
                                tool_used, reasoning, answer

                                like this:

                                Answer: answer
                                Tool Used: tool_name
                                Reasoning: reasoning for using the tool

                                An example:

                                Answer: 21.0
                                Tool Used: multiply
                                Reasoning: The tool was used to calculate the product of two numbers.


                                Solve the queries STEP by STEP and feel free to use the tools available to you and do not hallucinate or make assumptions.
                                """
        self.agent = self.get_agent()

    def get_agent(self):
        agent = FunctionCallingAgent.from_tools([multiply_tool, add_tool,
                                                 real_number_tool],
                                        llm=self.llm,verbose=True,
                                        system_prompt=self.system_prompt)
        return agent

    def chat(self, query: str):
        response_obj = self.agent.chat(query)
        return response_obj

In [6]:
agent = AgentController()
query = "what is 2+2"
agent.chat(query)

> Running step 07b60743-b95d-4054-b356-151a628c920d. Step input: what is 2+2
Added user message to memory: what is 2+2
=== LLM Response ===
Answer: 4.0
Tool Used: add
Reasoning: The tool was used to calculate the sum of two numbers.


AgentChatResponse(response='Answer: 4.0\nTool Used: add\nReasoning: The tool was used to calculate the sum of two numbers.', sources=[], source_nodes=[], is_dummy_stream=False, metadata=None)